<a href="https://colab.research.google.com/github/CP200435/-_TEAM_9124/blob/main/%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A_TEAM_9124.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [17]:
#下載資料集
import gdown
import os
import shutil

#下載training_image.zip
# gdown.download("https://drive.google.com/uc?export=download&id=1FfrsOgeJKSHtRM3Q649HJvSF1l4blxpl","/content/training_image.zip")
gdown.download("https://drive.google.com/uc?export=download&id=15jWkS3l4PFqvKAL9DW9DNzl94p4s3P_6","/content/training_image.zip")

#下載training_label.zip
# gdown.download("https://drive.google.com/uc?export=download&id=1KmYS2JhAexpWWlzlo_qwoX1TifA1COWW","/content/training_label.zip")
gdown.download("https://drive.google.com/uc?export=download&id=1K4n7G8zVK0_PtgkkAQFBEXJs-zJpElKG","/content/training_label.zip")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=15jWkS3l4PFqvKAL9DW9DNzl94p4s3P_6
From (redirected): https://drive.google.com/uc?export=download&id=15jWkS3l4PFqvKAL9DW9DNzl94p4s3P_6&confirm=t&uuid=c428df57-5379-41ed-b329-3f10cad67766
To: /content/training_image.zip
 31%|███       | 563M/1.83G [00:09<00:14, 86.2MB/s]

KeyboardInterrupt: 

In [ ]:
from textwrap import dedent
from pathlib import Path

data_yaml = dedent(
    """
train: "./datasets/train/images"
val: "./datasets/val/images"
test: "./datasets/test/images"

names:
  0: aortic_valve
"""
).strip()

config_path = Path("aortic_valve_colab.yaml")
config_path.write_text(data_yaml + "\n", encoding="utf-8")

print(f"YAML saved to: {config_path.resolve()}")
print(data_yaml)


In [ ]:
#下載資料集
import gdown
import os
!mkdir ./datasets
!mkdir ./datasets/test
gdown.download("https://drive.google.com/uc?export=download&id=13y81TV-z4_fbm7fCmjb0w_FvnPEZIHHa","/content/datasets/testing.zip")
!unzip '/content/datasets/testing' -d '/content/datasets/test/tmp'

In [ ]:
import os, zipfile, shutil  #week10hw

def unzip_if_needed(zip_path, dest_dir):
    if os.path.isdir(dest_dir):
        return

    if os.path.exists(zip_path):
        os.makedirs(dest_dir, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(dest_dir)

def find_patient_root(root):
    for dirpath, dirnames, _ in os.walk(root):
        if any(d.startswith("patient") for d in dirnames):
            return dirpath
    return root

# * 解壓縮 (不使用外部 unzip 指令)
unzip_if_needed("training_image.zip", "../training_image")
unzip_if_needed("training_label.zip", "../training_label")

IMG_ROOT = find_patient_root("../training_image")
LBL_ROOT = find_patient_root("../training_label")

# * 建立輸出資料夾
for p in ["./datasets/train/images", "./datasets/train/labels",
          "./datasets/val/images", "./datasets/val/labels",
          "./datasets/test/images", "./datasets/test/labels"]:
    os.makedirs(p, exist_ok=True)

In [ ]:
#week10
def move_patients(start, end, split):
    # train/val 讀前幾筆病患，後面當作 test 保留完整影像
    moved = 0

    for i in range(start, end + 1):
        patient = f"patient{i:04d}"
        img_dir = os.path.join(IMG_ROOT, patient)
        lbl_dir = os.path.join(LBL_ROOT, patient)
        if not os.path.isdir(img_dir):
            continue

        if split == "test":
            label_lookup = {}
            if os.path.isdir(lbl_dir):
                for frame in os.listdir(lbl_dir):
                    if frame.endswith(".txt"):
                        label_lookup[os.path.splitext(frame)[0]] = os.path.join(lbl_dir, frame)

            for frame in os.listdir(img_dir):
                if not frame.lower().endswith(".png"):
                    continue

                base = os.path.splitext(frame)[0]
                img_path = os.path.join(img_dir, frame)
                dst_img = f"./datasets/{split}/images/{base}.png"

                if os.path.exists(dst_img):
                    os.remove(dst_img)
                shutil.move(img_path, dst_img)

                lbl_path = label_lookup.get(base)
                if lbl_path and os.path.exists(lbl_path):
                    dst_lbl = f"./datasets/{split}/labels/{base}.txt"
                    if os.path.exists(dst_lbl):
                        os.remove(dst_lbl)
                    shutil.move(lbl_path, dst_lbl)

                moved += 1

        else:
            if not os.path.isdir(lbl_dir):
                continue

            for lbl_name in os.listdir(lbl_dir):
                if not lbl_name.endswith(".txt"):
                    continue

                base = os.path.splitext(lbl_name)[0]
                img_path = os.path.join(img_dir, base + ".png")
                lbl_path = os.path.join(lbl_dir, lbl_name)
                if not os.path.exists(img_path):
                    continue

                dst_img = f"./datasets/{split}/images/{base}.png"
                dst_lbl = f"./datasets/{split}/labels/{base}.txt"

                if os.path.exists(dst_img):
                    os.remove(dst_img)
                shutil.move(img_path, dst_img)

                if os.path.exists(dst_lbl):
                    os.remove(dst_lbl)
                shutil.move(lbl_path, dst_lbl)

                moved += 1

    return moved


In [ ]:
#week10
n_train = move_patients(1, 40, "train")
n_val = move_patients(41, 45, "val")
n_test = move_patients(46, 50, "test")

print("完成移動: train {} | val {} | test {}".format(n_train, n_val, n_test))

In [18]:
print('訓練集圖片數量 : ',len(os.listdir("./datasets/train/images")))
print('訓練集標記數量 : ',len(os.listdir("./datasets/train/labels")))
print('驗證集圖片數量 : ',len(os.listdir("./datasets/val/images")))
print('驗證集標記數量 : ',len(os.listdir("./datasets/val/labels")))
print('測試集圖片數量 : ',len(os.listdir("./datasets/test/images")))
print('測試集標記數量 : ',len(os.listdir("./datasets/test/labels")))
print('總圖片數量: ',len(os.listdir("./datasets/train/images")) + len(os.listdir("./datasets/val/images")) + len(os.listdir("./datasets/test/images")))
print('總標記數量: ',len(os.listdir("./datasets/train/labels")) + len(os.listdir("./datasets/val/labels")) + len(os.listdir("./datasets/test/labels")))

訓練集圖片數量 :  2168
訓練集標記數量 :  2168
驗證集圖片數量 :  292
驗證集標記數量 :  292
測試集圖片數量 :  1817
測試集標記數量 :  327
總圖片數量:  4277
總標記數量:  2787


In [ ]:
from ultralytics import YOLO

model = YOLO('yolo12n.pt') #初次訓練使用YOLO官方的預訓練模型，如要使用自己的模型訓練可以將'yolo12n.pt'替換掉
results = model.train(data="./aortic_valve_colab.yaml",
            epochs=20, #跑幾個epoch
            batch=16, #batch_size
            imgsz=640, #圖片大小640*640
            device=0 #使用GPU進行訓練
            )

In [ ]:
import os
import shutil

base_root = "/content/datasets/test/tmp"
dst_root1 = "/content/datasets/test/images1"
dst_root2 = "/content/datasets/test/images2"

os.makedirs(dst_root1, exist_ok=True)
os.makedirs(dst_root2, exist_ok=True)

# 自動找到第一個「直屬子資料夾含 patient*」的目錄
patient_root = base_root
for dirpath, dirnames, _ in os.walk(base_root):
    if any(d.lower().startswith("patient") for d in dirnames):
        patient_root = dirpath
        break

# 收集所有圖片路徑（只看直屬的 patient 資料夾）
all_files = []
for patient_folder in os.listdir(patient_root):
    patient_path = os.path.join(patient_root, patient_folder)
    if os.path.isdir(patient_path) and patient_folder.lower().startswith("patient"):
        for fname in os.listdir(patient_path):
            if fname.lower().endswith(".png"):
                all_files.append(os.path.join(patient_path, fname))

# 按名稱排序並對半移動
all_files.sort()
half = len(all_files) // 2

for f in all_files[:half]:
    shutil.move(f, os.path.join(dst_root1, os.path.basename(f)))

for f in all_files[half:]:
    shutil.move(f, os.path.join(dst_root2, os.path.basename(f)))

print(f"來源根目錄：{patient_root}")
print(f"完成移動！總共 {len(all_files)} 張，前半 {half} 張到 images1，後半 {len(all_files)-half} 張到 images2")


In [21]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.pt') #請自行更改最新的best.pt檔路徑
results = model.predict(source="./datasets/test/images1/",
              save=True,
              imgsz=640,
              device=0
              )

串流輸出內容已截斷至最後 5000 行。
image 3313/8310 /content/datasets/test/images1/patient0060_0359.png: 640x640 (no detections), 12.4ms
image 3314/8310 /content/datasets/test/images1/patient0060_0360.png: 640x640 (no detections), 13.2ms
image 3315/8310 /content/datasets/test/images1/patient0060_0361.png: 640x640 (no detections), 13.1ms
image 3316/8310 /content/datasets/test/images1/patient0060_0362.png: 640x640 (no detections), 13.7ms
image 3317/8310 /content/datasets/test/images1/patient0060_0363.png: 640x640 (no detections), 13.4ms
image 3318/8310 /content/datasets/test/images1/patient0060_0364.png: 640x640 (no detections), 13.4ms
image 3319/8310 /content/datasets/test/images1/patient0060_0365.png: 640x640 (no detections), 14.1ms
image 3320/8310 /content/datasets/test/images1/patient0060_0366.png: 640x640 (no detections), 12.3ms
image 3321/8310 /content/datasets/test/images1/patient0060_0367.png: 640x640 (no detections), 14.2ms
image 3322/8310 /content/datasets/test/images1/patient0060_0368.png: 6

In [22]:
print(len(results))

8310


In [23]:
!mkdir ./predict_txt/
output_file = open('./predict_txt/images1.txt', 'w')
for i in range(len(results)):
    # 取得圖片檔名（不含副檔名）
    filename = results[i].path.split('/')[-1].split('.png')[0]

    # 取得預測框數量
    boxes = results[i].boxes
    box_num = len(boxes.cls.tolist())

    # 如果有預測框
    if box_num > 0:
        for j in range(box_num):
            # 提取資訊
            label = int(boxes.cls[j].item())  # 類別
            conf = boxes.conf[j].item()       # 信心度
            x1, y1, x2, y2 = boxes.xyxy[j].tolist()  # 邊界框座標

            # 建立一行資料
            line = f"{filename} {label} {conf:.4f} {int(x1)} {int(y1)} {int(x2)} {int(y2)}\n"
            output_file.write(line)

# 關閉輸出檔案
output_file.close()

In [24]:
import torch ,gc

# 刪除大型變數
del boxes,all_files,results
gc.collect()
torch.cuda.empty_cache()

 31%|███       | 563M/1.83G [07:11<16:10, 1.30MB/s]


In [27]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.pt')
results = model.predict(source="./datasets/test/images2/",
              save=True,
              imgsz=640,
              device=0
              )

串流輸出內容已截斷至最後 5000 行。
image 3313/8310 /content/datasets/test/images2/patient0086_0051.png: 640x640 (no detections), 13.4ms
image 3314/8310 /content/datasets/test/images2/patient0086_0052.png: 640x640 (no detections), 12.0ms
image 3315/8310 /content/datasets/test/images2/patient0086_0053.png: 640x640 (no detections), 17.8ms
image 3316/8310 /content/datasets/test/images2/patient0086_0054.png: 640x640 (no detections), 13.0ms
image 3317/8310 /content/datasets/test/images2/patient0086_0055.png: 640x640 (no detections), 12.5ms
image 3318/8310 /content/datasets/test/images2/patient0086_0056.png: 640x640 (no detections), 13.3ms
image 3319/8310 /content/datasets/test/images2/patient0086_0057.png: 640x640 (no detections), 13.0ms
image 3320/8310 /content/datasets/test/images2/patient0086_0058.png: 640x640 (no detections), 13.7ms
image 3321/8310 /content/datasets/test/images2/patient0086_0059.png: 640x640 (no detections), 12.0ms
image 3322/8310 /content/datasets/test/images2/patient0086_0060.png: 6

In [28]:
output_file = open('./predict_txt/images2.txt', 'w')
for i in range(len(results)):
    # 取得圖片檔名（不含副檔名）
    filename = results[i].path.split('/')[-1].split('.png')[0]

    # 取得預測框數量
    boxes = results[i].boxes
    box_num = len(boxes.cls.tolist())

    # 如果有預測框
    if box_num > 0:
        for j in range(box_num):
            # 提取資訊
            label = int(boxes.cls[j].item())  # 類別
            conf = boxes.conf[j].item()       # 信心度
            x1, y1, x2, y2 = boxes.xyxy[j].tolist()  # 邊界框座標

            # 建立一行資料
            line = f"{filename} {label} {conf:.4f} {int(x1)} {int(y1)} {int(x2)} {int(y2)}\n"
            output_file.write(line)

# 關閉輸出檔案
output_file.close()

In [29]:
file1 = "./predict_txt/images1.txt"
file2 = "./predict_txt/images2.txt"
output = "./predict_txt/merged.txt"

with open(output, "w", encoding="utf-8") as fout:
    for f in [file1, file2]:
        if os.path.exists(f):
            with open(f, "r", encoding="utf-8") as fin:
                fout.writelines(fin.readlines())

print(f"合併完成 -> {output}")


合併完成 -> ./predict_txt/merged.txt


In [37]:
import torch ,gc

# 刪除大型變數
del boxes,all_files,results
gc.collect()
torch.cuda.empty_cache()

NameError: name 'results' is not defined

In [39]:
!yolo detect val model=/content/runs/detect/train/weights/best.pt data=aortic_valve_colab.yaml split=test save_txt save_conf

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 34.8±14.5 MB/s, size: 108.8 KB)
val: Scanning /content/datasets/test/labels... 327 images, 1490 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1817/1817 1.4Kit/s 1.3s
val: New cache created: /content/datasets/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 114/114 4.4it/s 25.9s
                   all       1817        327      0.838       0.89      0.879      0.573
Speed: 1.4ms preprocess, 6.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [42]:
import os
import glob
import cv2
import numpy as np
import yaml
from pathlib import Path
from tqdm import tqdm

# ================= 設定區域 (根據你的描述設定) =================
# 圖片路徑 (假設在 datasets/test/images 或類似結構，請依實際情況修改)
IMAGES_DIR = 'datasets/test/images'
# Ground Truth 標籤路徑 (假設在 datasets/test/labels)
LABELS_DIR = 'datasets/test/labels'
# 預測結果標籤路徑
PRED_DIR = r'/content/runs/detect/val/labels'
# YAML 設定檔 (用來讀取類別名稱)
YAML_PATH = 'aortic_valve_colab.yaml'
# 輸出結果圖片的資料夾
OUTPUT_DIR = 'verification_output'
# 輸出報告的檔名
REPORT_FILE = 'summary_report.md'

# IoU 閾值 (大於此值視為預測正確)
IOU_THRESHOLD = 0.5
# 線條粗細 (依要求設為細線)
LINE_THICKNESS = 1
# ==========================================================

def calculate_iou(box1, box2):
    """計算兩個框的 IoU (Intersection over Union)"""
    # box格式: [x1, y1, x2, y2]
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection

    return intersection / union if union > 0 else 0

def yolo_to_pixel(yolo_box, img_w, img_h):
    """將 YOLO 格式 (cls, x, y, w, h) 轉為像素座標 (x1, y1, x2, y2)"""
    x_c, y_c, w, h = yolo_box
    x1 = int((x_c - w/2) * img_w)
    y1 = int((y_c - h/2) * img_h)
    x2 = int((x_c + w/2) * img_w)
    y2 = int((y_c + h/2) * img_h)
    return [x1, y1, x2, y2]

def read_labels(file_path):
    """讀取 label txt 檔案"""
    boxes = []
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            for line in f:
                parts = list(map(float, line.strip().split()))
                # parts: class, x, y, w, h, (conf optional)
                cls_id = int(parts[0])
                coords = parts[1:5]
                conf = parts[5] if len(parts) > 5 else 1.0
                boxes.append({'cls': cls_id, 'coords': coords, 'conf': conf})
    return boxes

def main():
    # 建立輸出目錄
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # 讀取類別名稱
    class_names = []
    try:
        with open(YAML_PATH, 'r', encoding='utf-8') as f:
            data = yaml.safe_load(f)
            class_names = data.get('names', [])
    except Exception as e:
        print(f"警告: 無法讀取 YAML ({e})，將使用數字 ID 代替。")

    # 獲取所有測試圖片
    # 支援 jpg, png, jpeg
    img_paths = []
    for ext in ['*.jpg', '*.png', '*.jpeg']:
        img_paths.extend(glob.glob(os.path.join(IMAGES_DIR, ext)))

    if not img_paths:
        print(f"錯誤: 在 {IMAGES_DIR} 找不到圖片，請檢查路徑設定。")
        return

    report_lines = []
    report_lines.append("# YOLO 模型驗證詳細報告\n")
    report_lines.append("| 檔名 | 狀態 | 詳情 | IoU |")
    report_lines.append("|---|---|---|---|")

    stats = {
        "total_images": 0,
        "total_gt_objects": 0,
        "total_pred_objects": 0,
        "correct_predictions": 0,
        "false_positives": 0, # 預測有，實際無
        "false_negatives": 0, # 實際有，預測無
        "duplicate_detections": 0 # 多框指同一物
    }

    print("開始處理圖片並生成驗證結果...")

    for img_path in tqdm(img_paths):
        stats["total_images"] += 1
        filename = os.path.basename(img_path)
        file_id = os.path.splitext(filename)[0]

        # 讀取圖片
        img = cv2.imread(img_path)
        if img is None: continue
        h, w = img.shape[:2]

        # 定義 Label 路徑
        gt_path = os.path.join(LABELS_DIR, file_id + '.txt')
        pred_path = os.path.join(PRED_DIR, file_id + '.txt')

        gt_boxes = read_labels(gt_path)
        pred_boxes = read_labels(pred_path)

        stats["total_gt_objects"] += len(gt_boxes)
        stats["total_pred_objects"] += len(pred_boxes)

        # 轉換座標為像素格式
        gt_pixels = [yolo_to_pixel(b['coords'], w, h) for b in gt_boxes]
        pred_pixels = [yolo_to_pixel(b['coords'], w, h) for b in pred_boxes]

        # 標記狀態
        gt_matched = [False] * len(gt_boxes)
        pred_matched = [False] * len(pred_boxes)

        file_issues = []

        # 1. 檢查重疊與匹配 (Greedy Match: 找 IoU 最大的)
        for i, p_box in enumerate(pred_pixels):
            best_iou = 0
            best_gt_idx = -1

            for j, g_box in enumerate(gt_pixels):
                # 這裡假設類別必須相同才算 match，若不需類別相同可拿掉 cls 判斷
                if pred_boxes[i]['cls'] == gt_boxes[j]['cls']:
                    iou = calculate_iou(p_box, g_box)
                    if iou > best_iou:
                        best_iou = iou
                        best_gt_idx = j

            if best_iou >= IOU_THRESHOLD:
                if not gt_matched[best_gt_idx]:
                    # 完美匹配
                    gt_matched[best_gt_idx] = True
                    pred_matched[i] = True
                    stats["correct_predictions"] += 1
                    report_lines.append(f"| {filename} | 正確預測 | Class {pred_boxes[i]['cls']} | {best_iou:.2f} |")
                else:
                    # 2. 檢查同一張圖片是否出現多個框指向同一物件
                    stats["duplicate_detections"] += 1
                    file_issues.append("重複偵測")
                    pred_matched[i] = True # 標記為已匹配(雖然是重複的)
                    report_lines.append(f"| {filename} | **重複偵測** | 指向同一個 GT (Index {best_gt_idx}) | {best_iou:.2f} |")
            else:
                # 4. 檢查是否被預測到但實際上並不存在 (False Positive)
                # 這裡暫時不標記，等跑完確認沒 match 到任何 GT 的才算
                pass

        # 檢查 False Positives (殘留未匹配的預測框)
        for i, matched in enumerate(pred_matched):
            if not matched:
                stats["false_positives"] += 1
                file_issues.append("誤報(FP)")
                report_lines.append(f"| {filename} | **誤報 (FP)** | 預測存在但無 GT | - |")

        # 3. 檢查是否實際上存在但沒有被預測到 (False Negative)
        for i, matched in enumerate(gt_matched):
            if not matched:
                stats["false_negatives"] += 1
                file_issues.append("漏測(FN)")
                report_lines.append(f"| {filename} | **漏測 (FN)** | GT 存在但未測出 | - |")

        # ================= 6. 繪圖 (視覺化) =================
        # 繪製 Ground Truth (綠色)
        for i, box in enumerate(gt_pixels):
            # 如果是被漏測的，加粗一點或是換個顏色提示? 這裡依要求維持細線
            color = (0, 255, 0) # Green
            cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, LINE_THICKNESS)
            label = f"GT: {class_names[gt_boxes[i]['cls']] if class_names else gt_boxes[i]['cls']}"
            cv2.putText(img, label, (box[0], box[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

        # 繪製 Prediction (紅色)
        for i, box in enumerate(pred_pixels):
            color = (0, 0, 255) # Red
            # 如果是重複偵測，畫成橘色區分? 依要求"明顯看出差別"，紅色vs綠色已足夠
            if pred_matched[i] and "重複偵測" in file_issues:
                 color = (0, 165, 255) # Orange for duplicate hint (optional)

            cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, LINE_THICKNESS)
            conf_val = pred_boxes[i]['conf']
            label = f"Pred: {conf_val:.2f}"
            # 為了不遮擋 GT 文字，Pred 文字畫在框的底部
            cv2.putText(img, label, (box[0], box[3]+12), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

        # 儲存圖片
        cv2.imwrite(os.path.join(OUTPUT_DIR, filename), img)

    # 寫入總結報告
    with open(REPORT_FILE, 'w', encoding='utf-8') as f:
        f.write("# YOLO 檢測結果摘要報告\n\n")
        f.write("## 統計數據\n")
        f.write(f"- **總圖片數**: {stats['total_images']}\n")
        f.write(f"- **總物件數 (GT)**: {stats['total_gt_objects']}\n")
        f.write(f"- **總預測數 (Pred)**: {stats['total_pred_objects']}\n")
        f.write("### 異常分析\n")
        f.write(f"- **正確預測 (TP)**: {stats['correct_predictions']}\n")
        f.write(f"- **漏測 (FN)** (實際上存在但沒測到): {stats['false_negatives']}\n")
        f.write(f"- **誤報 (FP)** (測到了但實際上不存在): {stats['false_positives']}\n")
        f.write(f"- **重複偵測** (多個框指向同一物件): {stats['duplicate_detections']}\n\n")
        f.write("## 詳細列表\n")
        for line in report_lines:
            f.write(line + "\n")

    print(f"\n完成！\n1. 圖片已儲存至: {OUTPUT_DIR}\n2. 報告已儲存至: {REPORT_FILE}")

if __name__ == "__main__":
    main()

開始處理圖片並生成驗證結果...


100%|██████████| 1817/1817 [00:30<00:00, 59.42it/s]


完成！
1. 圖片已儲存至: verification_output
2. 報告已儲存至: summary_report.md
